In [ ]:
!pip install -U sentence-transformers

!pip install datasets

In [ ]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)
print(embeddings)

In [4]:
from datasets import load_dataset

dataset = load_dataset("ag_news")

  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
dataset['train']

Dataset({
    features: ['text', 'label'],
    num_rows: 120000
})

In [6]:
embeddings_train = model.encode(dataset['train']['text'])

In [7]:
embeddings_test = model.encode(dataset['test']['text'])

In [8]:
embeddings_train.shape

(120000, 384)

In [10]:
import tensorflow as tf

In [11]:
y_train = tf.keras.utils.to_categorical(dataset['train']['label'])

In [12]:
y_train.shape

(120000, 4)

In [27]:
import tensorflow as tf

model_tf = tf.keras.Sequential(layers=[
    tf.keras.layers.Dense(128),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(64),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(64),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(4, activation='softmax')
])


optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)
loss = tf.keras.losses.CategoricalCrossentropy()


model_tf.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

reduce_lr =tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                               patience=10, min_lr=0.0000001, verbose=1)


In [28]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
model_tf.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
model_tf.fit(embeddings_train, y_train, validation_split=0.15, epochs=35, batch_size=128, callbacks=[reduce_lr])

Epoch 1/35
797/797 [==============================] - 4s 4ms/step - loss: 0.3407 - accuracy: 0.8861 - val_loss: 0.2934 - val_accuracy: 0.8946 - lr: 0.0100
Epoch 2/35
797/797 [==============================] - 3s 4ms/step - loss: 0.3019 - accuracy: 0.8973 - val_loss: 0.2873 - val_accuracy: 0.9008 - lr: 0.0100
Epoch 3/35
797/797 [==============================] - 3s 3ms/step - loss: 0.2901 - accuracy: 0.9011 - val_loss: 0.2744 - val_accuracy: 0.9037 - lr: 0.0100
Epoch 4/35
797/797 [==============================] - 3s 4ms/step - loss: 0.2803 - accuracy: 0.9039 - val_loss: 0.2950 - val_accuracy: 0.8977 - lr: 0.0100
Epoch 5/35
797/797 [==============================] - 3s 3ms/step - loss: 0.2731 - accuracy: 0.9064 - val_loss: 0.2712 - val_accuracy: 0.9058 - lr: 0.0100
Epoch 6/35
797/797 [==============================] - 3s 3ms/step - loss: 0.2693 - accuracy: 0.9076 - val_loss: 0.2846 - val_accuracy: 0.9022 - lr: 0.0100
Epoch 7/35
797/797 [==============================] - 3s 3ms/step - lo

In [29]:
y_test = tf.keras.utils.to_categorical(dataset['test']['label'])

In [30]:
model_tf.evaluate(embeddings_test, y_test)

238/238 [==============================] - 1s 2ms/step - loss: 0.2785 - accuracy: 0.9176


[0.27851951122283936, 0.9176315665245056]